# Predicting sentiment from product reviews


The goal of this first notebook is to explore logistic regression and feature engineering with existing GraphLab functions.

In this notebook you will use product review data from Amazon.com to predict whether the sentiments about a product (from its reviews) are positive or negative.

* Use SFrames to do some feature engineering
* Train a logistic regression model to predict the sentiment of product reviews.
* Inspect the weights (coefficients) of a trained logistic regression model.
* Make a prediction (both class and probability) of sentiment for a new product review.
* Given the logistic regression weights, predictors and ground truth labels, write a function to compute the **accuracy** of the model.
* Inspect the coefficients of the logistic regression model and interpret their meanings.
* Compare multiple logistic regression models.

Let's get started!
    
## Fire up GraphLab Create

Make sure you have the latest version of GraphLab Create.

In [1]:
from __future__ import division
import graphlab
import math
import string

# Data preparation

We will use a dataset consisting of baby product reviews on Amazon.com.

In [2]:
products = graphlab.SFrame('amazon_baby.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to angelasinger70446@gmail.com and will expire on August 19, 2020.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\angel\AppData\Local\Temp\graphlab_server_1567324680.log.0


Now, let us see a preview of what the dataset looks like.

In [3]:
products

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0


## Build the word count vector for each review

Let us explore a specific example of a baby product.


In [4]:
products[269]

{'name': 'The First Years Massaging Action Teether',
 'rating': 5.0,
 'review': 'A favorite in our house!'}

Now, we will perform 2 simple data transformations:

1. Remove punctuation using [Python's built-in](https://docs.python.org/2/library/string.html) string functionality.
2. Transform the reviews into word-counts.

**Aside**. In this notebook, we remove all punctuations for the sake of simplicity. A smarter approach to punctuations would preserve phrases such as "I'd", "would've", "hadn't" and so forth. See [this page](https://www.cis.upenn.edu/~treebank/tokenization.html) for an example of smart handling of punctuations.

In [5]:
def remove_punctuation(text):
    import string
    return text.translate(None, string.punctuation) 

review_without_punctuation = products['review'].apply(remove_punctuation)
products['word_count'] = graphlab.text_analytics.count_words(review_without_punctuation)

Now, let us explore what the sample example above looks like after these 2 transformations. Here, each entry in the **word_count** column is a dictionary where the key is the word and the value is a count of the number of times the word occurs.

In [6]:
products[269]['word_count']

{'a': 1L, 'favorite': 1L, 'house': 1L, 'in': 1L, 'our': 1L}

## Extract sentiments

We will **ignore** all reviews with *rating = 3*, since they tend to have a neutral sentiment.

In [7]:
products = products[products['rating'] != 3]
len(products)

166752

Now, we will assign reviews with a rating of 4 or higher to be *positive* reviews, while the ones with rating of 2 or lower are *negative*. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label.

In [8]:
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)
products

name,review,rating,word_count,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3L, 'love': 1L,'it': 3L, 'highly': 1L, ...",1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2L, 'quilt': 1L,'it': 1L, 'comfortable': ...",1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'and': 3L, 'ingenious':1L, 'love': 2L, 'is': ...",1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2L, 'all': 2L,'help': 1L, 'cried': 1L, ...",1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2L, 'cute': 1L,'help': 2L, 'habit': 1L, ...",1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1L, 'be': 1L,'is': 1L, 'bound': 1L, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'and': 2L, 'all': 1L,'right': 1L, 'able': 1L, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1L, 'fantastic':1L, 'help': 1L, 'give': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'all': 1L, 'standarad':1L, 'another': 1L, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'all': 2L, 'nannys': 1L,'just': 1L, 'sleep': 2L, ...",1


Now, we can see that the dataset contains an extra column called **sentiment** which is either positive (+1) or negative (-1).

## Split data into training and test sets

Let's perform a train/test split with 80% of the data in the training set and 20% of the data in the test set. We use `seed=1` so that everyone gets the same result.

In [9]:
train_data, test_data = products.random_split(.8, seed=1)
print len(train_data)
print len(test_data)

133416
33336


# Train a sentiment classifier with logistic regression

We will now use logistic regression to create a sentiment classifier on the training data. This model will use the column **word_count** as a feature and the column **sentiment** as the target. We will use `validation_set=None` to obtain same results as everyone else.

**Note:** This line may take 1-2 minutes.

In [10]:
sentiment_model = graphlab.logistic_classifier.create(train_data,
                                                      target = 'sentiment',
                                                      features=['word_count'],
                                                      validation_set=None)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133416

Number of classes           : 2

Number of feature columns   : 1

...

| 5         | 12       | 3.000000  | 5.267919     | 0.960964          |

| 6         | 13       | 3.000000  | 5.760595     | 0.975033          |

+-----------+----------+-----------+--------------+-------------------+

TERMINATED: Terminated due to numerical difficulties.

This model may not be ideal. To improve it, consider doing one of the following:
(a) Increasing the regularization.
(b) Standardizing the input data.
(c) Removing highly correlated features.
(d) Removing `inf` and `NaN` values in the training data.

In [11]:
sentiment_model

Class                          : LogisticClassifier

Schema
------
Number of coefficients         : 121713
Number of examples             : 133416
Number of classes              : 2
Number of feature columns      : 1
Number of unpacked features    : 121712

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : lbfgs
Solver iterations              : 6
Solver status                  : TERMINATED: Terminated due to numerical difficulties.
Training time (sec)            : 6.338

Settings
--------
Log-likelihood                 : inf

Highest Positive Coefficients
-----------------------------
word_count[mobileupdate]       : 41.9847
word_count[placeid]            : 41.7354
word_count[labelbox]           : 41.151
word_count[httpwwwamazoncomreviewrhgg6qp7tdnhbrefcmcrprcmtieutf8asinb00318cla0nodeid] : 40.0454
word_count[knobskeeping]       : 36.2091

Lowest Negative Coeffic

**Aside**. You may get a warning to the effect of "Terminated due to numerical difficulties --- this model may not be ideal". It means that the quality metric (to be covered in Module 3) failed to improve in the last iteration of the run. The difficulty arises as the sentiment model puts too much weight on extremely rare words. A way to rectify this is to apply regularization, to be covered in Module 4. Regularization lessens the effect of extremely rare words. For the purpose of this assignment, however, please proceed with the model above.

Now that we have fitted the model, we can extract the weights (coefficients) as an SFrame as follows:

In [12]:
weights = sentiment_model.coefficients
weights.column_names()

['name', 'index', 'class', 'value', 'stderr']

There are a total of `121713` coefficients in the model. Recall from the lecture that positive weights $w_j$ correspond to weights that cause positive sentiment, while negative weights correspond to negative sentiment. 

Fill in the following block of code to calculate how many *weights* are positive ( >= 0). (**Hint**: The `'value'` column in SFrame *weights* must be positive ( >= 0)).

In [17]:
num_positive_weights = 0
num_negative_weights = 0

print(len(weights))


for i in range(len(weights)):
    print(weights["value"][i])
    if (weights["value"][i] >=0):
        num_positive_weights +=1
    else:
        num_negative_weights +=1
        


print "Number of positive weights: %s " % num_positive_weights
print "Number of negative weights: %s " % num_negative_weights

121713
1.30337080544
0.303815600015
1.49183015276
-3.95748618393
1.43301685439
0.00986646490307
-0.797764553926
0.048449573172
0.165541436615
-0.0949937947269
-0.817151716331
0.488413478808
0.131378480765
0.0182528116279
-0.0769014155235
-0.302595277297
0.135472678413
-0.0530004786379
0.0383787882079
0.165506649337
-0.0300523581013
-0.24658014554
7.3963370872
0.426326525702
0.671556821414
0.974344638785
0.0529359292783
0.0928734488917
0.130090528525
0.0108336553052
2.39234034193
-0.0227274977677
0.121909799418
0.249426452207
0.0404928422711
0.908512104908
0.0778583404287
0.0967118520361
2.39234034193
0.643215125748
-0.299515909175
0.11969744113
0.246087063318
-0.00410088996429
0.567749125472
0.883769830003
-0.10300729812
-1.63495562697
0.197472691417
0.225401372518
-0.339206253047
0.165368635022
0.985252010757
-0.0331634234839
0.0726193969013
-0.228603781625
0.63874418563
0.209048733253
0.114674517638
2.10052574895
0.278525110913
0.0908797403005
0.891364020759
0.627964877567
0.21028110

0.050508759814
0.0700931697384
0.193543350553
-0.276920953578
1.15613703596
1.03685354088
0.104092312599
-0.1796191709
0.109869611367
-1.82656180069
0.65571907734
-0.251581668813
-0.0916746745268
0.0210172686347
-0.013510144273
0.345997782638
-0.241498411367
1.70608001765
2.96127680229
0.394517193171
0.0596010389788
1.04030830683
0.100466150455
-1.69933935877
0.528689255995
-1.62985546527
-0.0627353857048
0.272697577448
-0.88483971859
-2.35213711506
1.19550456484
-0.0118494250363
0.733478769123
-0.290180426597
0.681697834661
1.25240704921
-0.118818881942
-0.0147930074345
0.100466150455
0.303441110726
1.59233992925
-0.416512480248
-0.459838986175
0.632595563307
0.267682254773
0.100392226378
0.104593609635
0.653724934274
1.25896509755
1.88962115818
-0.131429702697
0.631133668648
0.269435842145
0.443646964714
-0.105779956819
0.769555505613
0.0130919025047
1.93617282332
0.684793784582
-0.0981114349338
0.769555505613
-0.365979650675
1.51399723974
1.29274788645
0.420753070364
-0.443537896396

-0.105447110605
4.22145773942
-1.02232441527
-0.0236712836726
-0.0642037219806
0.468468400242
1.39261516586
0.1646385742
0.210007375389
-0.546387703925
0.0608735590423
0.141618918205
0.261223409887
-0.340507448004
0.398221725523
-1.50598186842
1.59915610172
0.896528324605
0.0610687597583
-0.0383893029508
0.0196302335221
-0.0420399104832
-0.104499008372
0.238945058573
0.424827945877
2.46016967434
0.6477216264
-0.914408400766
0.458130143207
-0.704001097949
0.348302301676
-3.47719264826
-0.303792406523
2.43921836531
0.659208739909
1.82312206204
-0.715393498301
0.337004161776
0.215890303688
-0.17572778423
-1.03645842134
-0.467674805022
0.488510251255
0.969802594031
0.000710203841907
-0.238227290639
-0.152453607671
0.0953142395436
-1.27832376309
-0.528622906289
0.737124999426
-0.346998530714
0.0538319271429
-0.975772929029
6.74885024332
0.133991373414
-0.701677005292
-0.317707975617
-0.0194697204202
0.439423043816
-0.236530694229
1.05270782849
-0.0764691290958
0.0928926063092
1.33471619946


0.142746230812
-3.12535080633
-0.053632109793
0.0330168285003
-4.0741997247
-6.37183467111
1.20000096964
-4.11907951274
1.15237875354
3.64204152764
-0.0898546196466
0.388573786144
0.449403788592
1.17418387343
0.514056810872
0.966078257332
0.809407801159
-1.00003973113
2.36850062129
-0.115536883256
-0.0224636549116
-0.581837755479
0.431270913957
0.195745865411
0.503545125705
-0.245068079565
-0.854718041118
0.217768571298
0.221288754313
1.19882452521
-1.06969463704
0.00738631750863
-0.0193928925307
0.855787580497
-1.32546302476
1.23211651531
0.30535380911
1.24911229193
0.161112551358
0.143221699061
-0.17023873558
0.177165661643
0.346339663603
0.0332922234752
-0.905617723863
0.198466958853
1.17175038402
0.325454488043
0.604202764621
-1.33740902067
1.55711760427
3.50387306638
2.36565779413
0.771656333046
2.36565779413
-0.076925539197
-4.14201225499
0.40995786906
-0.165480291115
0.484506142004
-0.283592898896
0.389966934068
0.418937255394
0.0060551641346
0.444994816867
0.883804343974
1.7060

-0.959653317988
0.0650130100652
1.28426387027
-0.17902309492
1.94163005598
-0.390378745008
-0.335001780943
-0.0737444242253
-0.295678083352
-1.08741615954
0.132012327969
-0.417083932988
0.334970622227
0.845040963194
0.405660421802
-1.73224319343
0.298634491393
0.634937770932
-2.4123831985
-0.280173902103
-0.0244991010721
0.930578150582
-1.288484302
-0.452701774857
-11.1460876234
0.343614349435
-2.41209050909
-1.10365483261
-0.134380924808
0.0991022737899
0.596006832045
-3.04152603562
-1.70109878369
-0.00919640761786
1.43345121555
1.50870126134
0.697952589689
0.19773543501
-0.37238503177
0.566997127932
0.252252627242
0.0332342266875
0.0175711407195
-0.69458448122
1.44795534154
2.11963655635
0.971978428782
-0.308543749826
-8.41832035389
0.568074509808
-1.73397526272
0.0159563939875
-0.263716991395
-0.0399044456465
-2.64998410346
0.632772443276
0.077628665029
-1.38318451342
-0.420667781484
0.112704216595
0.952591045339
0.775465320052
-0.0811047671667
-2.46766348673
0.321820152501
0.834273

...

-4.01586300696
-4.01586300696
5.37011731982
7.72539170011
4.16262300007
7.72539170011
3.6901981875
-8.92838643396
2.64454168235
2.64454168235
-5.91378803103
-5.71169952361
-5.71169952361
-0.90501405296
6.22582858033
6.22582858033
-8.81385879028
-4.57782539507
-4.57782539507
-7.3850100377
0.976300779297
0.976300779297
0.976300779297
1.74483633757
1.74483633757
1.0150020653
1.0150020653
1.0150020653
2.23826533551
2.23826533551
-0.119483608276
-0.119483608276
-0.119483608276
-0.119483608276
-0.119483608276
-0.119483608276
-0.119483608276
3.50134274477
3.31121626998
3.31121626998
13.0396269375
-5.7948286999
-5.7948286999
7.26059508197
1.23224934311
1.23224934311
1.23224934311
1.23224934311
4.36962423882
0.94702049202
1.88374318515
1.88374318515
-5.51049852584
-8.53640989772
1.70595249988
1.70595249988
0.846017385378
0.846017385378
-7.88182885464
2.26718909306
2.26718909306
-7.49160604113
3.23358862647
-6.13184224317
-6.13184224317
1.92981963502
1.92981963502
-9.16633644208
-3.47453749141
-

2.04437062117
2.04437062117
2.04437062117
2.04437062117
-5.03836823383
-5.03836823383
-5.03836823383
0.248453892863
0.248453892863
1.18846981237
1.15333864241
-0.0387785185724
-0.0387785185724
-0.0387785185724
-0.0387785185724
0.118057964053
-5.83285196023
-7.27877151477
0.922226885433
4.59381885093
11.0343699403
4.68700444682
1.68222389672
-10.7676610873
-8.61387330707
-8.61387330707
-8.61387330707
-8.61387330707
0.697940148751
0.697940148751
1.07483147433
0.982368986202
0.982368986202
3.54128161049
3.54128161049
-8.98396041089
-11.5616388645
-5.27412117088
-5.27412117088
0.843213313673
0.998051934805
0.998051934805
0.0905491788018
0.0905491788018
0.0905491788018
0.0905491788018
0.0905491788018
0.0905491788018
0.0359623370892
0.0359623370892
1.03691192882
0.630409194265
0.661472159666
0.661472159666
-0.189320408703
-0.189320408703
2.35088665089
-6.04382170035
-6.04382170035
0.687628087354
0.687628087354
0.687628087354
0.687628087354
0.687628087354
0.654278797213
-8.12016767619
0.01449

-6.57665852555
0.0965354876111
0.0965354876111
0.0965354876111
0.0965354876111
2.23925690872
1.67486157842
1.67486157842
1.67486157842
1.95038266658
2.34827089962
2.34827089962
2.34827089962
2.34827089962
2.34827089962
1.39715539421
1.85321588346
3.81154479315
0.109105337645
0.109105337645
0.301949535564
1.95637235148
-0.112519380827
1.53868043721
0.248747184889
0.248747184889
-0.0738894011044
2.41218766981
0.6675331399
0.562970415096
0.815681589067
0.815681589067
0.815681589067
0.815681589067
0.815681589067
0.815681589067
0.815681589067
0.820933657841
-0.13544110557
-0.13544110557
-0.108649296648
0.165338125356
2.32011005399
2.32011005399
2.32011005399
2.32011005399
2.32011005399
2.32011005399
0.12748342543
0.12748342543
1.9137017509
0.671099606229
0.671099606229
0.463676362338
0.463676362338
0.451105808916
2.95303751213
2.95303751213
-0.116933417125
3.18634701769
-0.291724489338
-0.291724489338
-0.291724489338
-0.291724489338
-0.291724489338
0.322891134522
1.37540255874
0.19040195663

0.472473214819
0.472473214819
1.16125824048
2.6732469645
3.75918210776
8.64790984027
8.64790984027
-0.30876396645
-0.30876396645
-0.30876396645
-0.30876396645
-0.30876396645
-0.30876396645
-0.30876396645
-0.30876396645
-0.30876396645
-0.30876396645
2.67276815069
30.4162931822
30.4162931822
30.4162931822
30.4162931822
-0.292909998365
-0.292909998365
1.91298802208
1.89705723185
1.89705723185
-0.210137691407
-0.307229768579
-0.307229768579
-0.307229768579
-0.307229768579
-0.307229768579
-0.307229768579
-0.307229768579
-0.307229768579
-0.307229768579
-0.307229768579
-0.279524919032
-0.279524919032
-0.279524919032
-0.279524919032
-0.279524919032
-0.279524919032
-0.279524919032
-0.279524919032
-0.279524919032
-0.279524919032
-0.279524919032
-0.272883664306
-0.272883664306
-0.272883664306
-0.258925571113
0.214794607401
-0.159162641572
-0.159162641572
-0.159162641572
-0.159162641572
-0.159162641572
-0.159162641572
-0.159162641572
0.429635065125
-0.309553925014
-0.309553925014
-0.309553925014
-

7.62879015404
5.27947680447
2.92881278673
6.16152891431
1.35139667293
1.35139667293
1.25283133241
-5.3438452184
-6.81193936086
2.1638163399
0.154600585972
0.0773002929862
8.11289593828
1.77689222094
1.77689222094
0.495096682055
-10.2775237889
0.402855954025
6.95403123987
0.733365097683
0.94984000718
0.94984000718
0.94984000718
4.46797279921
-0.0174926242966
8.15155784435
-6.33743982944
-6.33743982944
-7.98530111841
0.119511019041
0.119511019041
2.07228805598
-8.11870707116
2.00283682934
5.38078575566
5.38078575566
1.17389202417
1.17389202417
0.586946012084
3.02042665345
4.33567344378
3.48222014569
2.28810400162
7.56345581484
7.56345581484
7.56345581484
-5.40131788993
1.94624617413
2.63256775906
-6.31089779777
2.16385062032
6.68993307355
-3.88297012175
0.118649639519
0.118649639519
0.118649639519
0.118649639519
-9.31942428033
8.25829206605
1.74535324016
1.80011657038
0.889673606116
2.01272511145
0.827831678957
0.827831678957
-7.47767685263
-7.47767685263
3.23396862032
0.799219685253
0.7

**Quiz Question:** How many weights are >= 0?

## Making predictions with logistic regression

Now that a model is trained, we can make predictions on the **test data**. In this section, we will explore this in the context of 3 examples in the test dataset.  We refer to this set of 3 examples as the **sample_test_data**.

In [18]:
sample_test_data = test_data[10:13]
print sample_test_data['rating']
sample_test_data

[5.0, 2.0, 1.0]


name,review,rating,word_count,sentiment
Our Baby Girl Memory Book,Absolutely love it andall of the Scripture in ...,5.0,"{'and': 2L, 'all': 1L,'love': 1L, ...",1
Wall Decor RemovableDecal Sticker - Colorful ...,Would not purchase againor recommend. The decals ...,2.0,"{'and': 1L, 'wall': 1L,'them': 1L, 'decals': ...",-1
New Style Trailing CherryBlossom Tree Decal ...,Was so excited to getthis product for my baby ...,1.0,"{'all': 1L, 'money': 1L,'into': 1L, 'it': 3L, ...",-1


Let's dig deeper into the first row of the **sample_test_data**. Here's the full review:

In [19]:
sample_test_data[0]['review']

'Absolutely love it and all of the Scripture in it.  I purchased the Baby Boy version for my grandson when he was born and my daughter-in-law was thrilled to receive the same book again.'

That review seems pretty positive.

Now, let's see what the next row of the **sample_test_data** looks like. As we could guess from the sentiment (-1), the review is quite negative.

In [20]:
sample_test_data[1]['review']

'Would not purchase again or recommend. The decals were thick almost plastic like and were coming off the wall as I was applying them! The would NOT stick! Literally stayed stuck for about 5 minutes then started peeling off.'

We will now make a **class** prediction for the **sample_test_data**. The `sentiment_model` should predict **+1** if the sentiment is positive and **-1** if the sentiment is negative. Recall from the lecture that the **score** (sometimes called **margin**) for the logistic regression model  is defined as:

$$
\mbox{score}_i = \mathbf{w}^T h(\mathbf{x}_i)
$$ 

where $h(\mathbf{x}_i)$ represents the features for example $i$.  We will write some code to obtain the **scores** using GraphLab Create. For each row, the **score** (or margin) is a number in the range **[-inf, inf]**.

In [21]:
scores = sentiment_model.predict(sample_test_data, output_type='margin')
print scores

[6.73461972705967, -5.734130996760552, -14.66846040446904]


### Predicting sentiment

These scores can be used to make class predictions as follows:

$$
\hat{y} = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{w}^T h(\mathbf{x}_i) > 0 \\
      -1 & \mathbf{w}^T h(\mathbf{x}_i) \leq 0 \\
\end{array} 
\right.
$$

Using scores, write code to calculate $\hat{y}$, the class predictions:

In [28]:
predictions = list()
for i in range(len(scores)):
    if scores[i] > 0:
        predictions.append(1)
    else:
        predictions.append(-1)
        
print(predictions)

[1, -1, -1]


Run the following code to verify that the class predictions obtained by your calculations are the same as that obtained from GraphLab Create.

In [29]:
print "Class predictions according to GraphLab Create:" 
print sentiment_model.predict(sample_test_data)

Class predictions according to GraphLab Create:
[1L, -1L, -1L]


**Checkpoint**: Make sure your class predictions match with the one obtained from GraphLab Create.

### Probability predictions

Recall from the lectures that we can also calculate the probability predictions from the scores using:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))}.
$$

Using the variable **scores** calculated previously, write code to calculate the probability that a sentiment is positive using the above formula. For each row, the probabilities should be a number in the range **[0, 1]**.

In [33]:
probs = list()
for i in range(len(scores)):
    probs.append(1/(1+math.exp(-scores[i])))
    
print(probs)

[0.9988123848377203, 0.003223268181799736, 4.2615579966532025e-07]


**Checkpoint**: Make sure your probability predictions match the ones obtained from GraphLab Create.

In [48]:
print "Class predictions according to GraphLab Create:" 
print sentiment_model.predict(sample_test_data, output_type='probability')

Class predictions according to GraphLab Create:
[1L, -1L, -1L]


** Quiz Question:** Of the three data points in **sample_test_data**, which one (first, second, or third) has the **lowest probability** of being classified as a positive review?

# Find the most positive (and negative) review

We now turn to examining the full test dataset, **test_data**, and use GraphLab Create to form predictions on all of the test data points for faster performance.

Using the `sentiment_model`, find the 20 reviews in the entire **test_data** with the **highest probability** of being classified as a **positive review**. We refer to these as the "most positive reviews."

To calculate these top-20 reviews, use the following steps:
1.  Make probability predictions on **test_data** using the `sentiment_model`. (**Hint:** When you call `.predict` to make predictions on the test data, use option `output_type='probability'` to output the probability rather than just the most likely class.)
2.  Sort the data according to those predictions and pick the top 20. (**Hint:** You can use the `.topk` method on an SFrame to find the top k rows sorted according to the value of a specified column.)

In [44]:
prediction = sentiment_model.predict(test_data, output_type="probability")
test_data["prob1"] = prediction

best20 = test_data.topk("prob1", k=20)
best20.print_rows(num_rows=20)

+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
| Britax Decathlon Convertib... | I researched a few differe... |  4.0   |
| Ameda Purely Yours Breast ... | As with many new moms, I c... |  4.0   |
| Traveling Toddler Car Seat... | I am sure this product wor... |  2.0   |
| Shermag Glider Rocker Comb... | After searching in stores ... |  4.0   |
| Cloud b Sound Machine Soot... | First off, I love plush sh... |  5.0   |
| JP Lizzy Chocolate Ice Cla... | I got this bag as a presen... |  4.0   |
| Fisher-Price Rainforest Me... | My daughter wasn't able to... |  5.0   |
| Lilly Gold Sit 'n' Stroll ... | I just completed a two-mon... |  5.0   |
|  Fisher-Price Deluxe Jumperoo | I had already decided that... |  5.0   |
| North States Supergate Pre... | I got a couple of these to... |  4.0   |
|   Munchkin Mozart Magic

**Quiz Question**: Which of the following products are represented in the 20 most positive reviews? [multiple choice]


Now, let us repeat this exercise to find the "most negative reviews." Use the prediction probabilities to find the  20 reviews in the **test_data** with the **lowest probability** of being classified as a **positive review**. Repeat the same steps above but make sure you **sort in the opposite order**.

In [47]:
worst20 = test_data.topk("prob1", k=20, reverse = True)
worst20.print_rows(num_rows=20)


+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
| Jolly Jumper Arctic Sneak ... | I am a "research-aholic" i... |  5.0   |
| Levana Safe N'See Digital ... | This is the first review I... |  1.0   |
| Snuza Portable Baby Moveme... | I would have given the pro... |  1.0   |
| Fisher-Price Ocean Wonders... | We have not had ANY luck w... |  2.0   |
| VTech Communications Safe ... | This is my second video mo... |  1.0   |
| Safety 1st High-Def Digita... | We bought this baby monito... |  1.0   |
| Chicco Cortina KeyFit 30 T... | My wife and I have used th... |  1.0   |
| Prince Lionheart Warmies W... | *****IMPORTANT UPDATE*****... |  1.0   |
| Valco Baby Tri-mode Twin S... | I give one star to the dim... |  1.0   |
| Adiri BPA Free Natural Nur... | I will try to write an obj... |  2.0   |
| Munchkin Nursery Projec

**Quiz Question**: Which of the following products are represented in the 20 most negative reviews?  [multiple choice]

## Compute accuracy of the classifier

We will now evaluate the accuracy of the trained classifier. Recall that the accuracy is given by


$$
\mbox{accuracy} = \frac{\mbox{# correctly classified examples}}{\mbox{# total examples}}
$$

This can be computed as follows:

* **Step 1:** Use the trained model to compute class predictions (**Hint:** Use the `predict` method)
* **Step 2:** Count the number of data points when the predicted class labels match the ground truth labels (called `true_labels` below).
* **Step 3:** Divide the total number of correct predictions by the total number of data points in the dataset.

Complete the function below to compute the classification accuracy:

In [49]:
def get_classification_accuracy(model, data, true_labels):
    prediction = model.predict(data, output_type = "class" )
    
    # Compute the number of correctly classified examples
    ## YOUR CODE HERE
    num_correct = 0
    for i in range(len(data)):
        if true_labels[i] == prediction[i]:
            num_correct += 1

    # Then compute accuracy by dividing num_correct by total number of examples
    ## YOUR CODE HERE
    accuracy = num_correct/len(true_labels)
    
    return accuracy

Now, let's compute the classification accuracy of the **sentiment_model** on the **test_data**.

In [50]:
get_classification_accuracy(sentiment_model, test_data, test_data['sentiment'])

0.9145368370530358

**Quiz Question**: What is the accuracy of the **sentiment_model** on the **test_data**? Round your answer to 2 decimal places (e.g. 0.76).

**Quiz Question**: Does a higher accuracy value on the **training_data** always imply that the classifier is better?

## Learn another classifier with fewer words

There were a lot of words in the model we trained above. We will now train a simpler logistic regression model using only a subset of words that occur in the reviews. For this assignment, we selected a 20 words to work with. These are:

In [51]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [52]:
len(significant_words)

20

For each review, we will use the **word_count** column and trim out all words that are **not** in the **significant_words** list above. We will use the [SArray dictionary trim by keys functionality]( https://dato.com/products/create/docs/generated/graphlab.SArray.dict_trim_by_keys.html). Note that we are performing this on both the training and test set.

In [53]:
train_data['word_count_subset'] = train_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)
test_data['word_count_subset'] = test_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)

Let's see what the first example of the dataset looks like:

In [54]:
train_data[0]['review']

'it came early and was not disappointed. i love planet wise bags and now my wipe holder. it keps my osocozy wipes moist and does not leak. highly recommend it.'

The **word_count** column had been working with before looks like the following:

In [55]:
print train_data[0]['word_count']

{'and': 3L, 'love': 1L, 'it': 3L, 'highly': 1L, 'osocozy': 1L, 'bags': 1L, 'leak': 1L, 'moist': 1L, 'does': 1L, 'recommend': 1L, 'was': 1L, 'wipes': 1L, 'disappointed': 1L, 'early': 1L, 'not': 2L, 'now': 1L, 'holder': 1L, 'wipe': 1L, 'keps': 1L, 'wise': 1L, 'i': 1L, 'planet': 1L, 'my': 2L, 'came': 1L}


Since we are only working with a subset of these words, the column **word_count_subset** is a subset of the above dictionary. In this example, only 2 `significant words` are present in this review.

In [56]:
print train_data[0]['word_count_subset']

{'love': 1L, 'disappointed': 1L}


## Train a logistic regression model on a subset of data

We will now build a classifier with **word_count_subset** as the feature and **sentiment** as the target. 

In [57]:
simple_model = graphlab.logistic_classifier.create(train_data,
                                                   target = 'sentiment',
                                                   features=['word_count_subset'],
                                                   validation_set=None)
simple_model

Logistic regression:

--------------------------------------------------------

Number of examples          : 133416

Number of classes           : 2

Number of feature columns   : 1

...

| 5         | 6        | 0.693151     | 0.866815          |

| 6         | 7        | 0.813825     | 0.866815          |

+-----------+----------+--------------+-------------------+

SUCCESS: Optimal solution found.

Class                          : LogisticClassifier

Schema
------
Number of coefficients         : 21
Number of examples             : 133416
Number of classes              : 2
Number of feature columns      : 1
Number of unpacked features    : 20

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : newton
Solver iterations              : 6
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 0.8417

Settings
--------
Log-likelihood                 : 44323.7254

Highest Positive Coefficients
-----------------------------
word_count_subset[loves]       : 1.6773
word_count_subset[perfect]     : 1.5145
word_count_subset[love]        : 1.3654
(intercept)                    : 1.2995
word_count_subset[easy]        : 1.1937

Lowest Negative Coefficients
----------------------------
word_count_subset[disappointed] : -2.3551
wo

We can compute the classification accuracy using the `get_classification_accuracy` function you implemented earlier.

In [58]:
get_classification_accuracy(simple_model, test_data, test_data['sentiment'])

0.8693004559635229

Now, we will inspect the weights (coefficients) of the **simple_model**:

In [59]:
simple_model.coefficients

name,index,class,value,stderr
(intercept),None,1,1.2995449552,0.0120888541331
word_count_subset,disappointed,1,-2.35509250061,0.0504149888557
word_count_subset,love,1,1.36543549368,0.0303546295109
word_count_subset,well,1,0.504256746398,0.021381300631
word_count_subset,product,1,-0.320555492996,0.0154311321362
word_count_subset,loves,1,1.67727145556,0.0482328275384
word_count_subset,little,1,0.520628636025,0.0214691475665
word_count_subset,work,1,-0.621700012425,0.0230330597946
word_count_subset,easy,1,1.19366189833,0.029288869202
word_count_subset,great,1,0.94469126948,0.0209509926591


Let's sort the coefficients (in descending order) by the **value** to obtain the coefficients with the most positive effect on the sentiment.

In [60]:
simple_model.coefficients.sort('value', ascending=False).print_rows(num_rows=21)

+-------------------+--------------+-------+-----------------+-----------------+
|        name       |    index     | class |      value      |      stderr     |
+-------------------+--------------+-------+-----------------+-----------------+
| word_count_subset |    loves     |   1   |  1.67727145556  | 0.0482328275384 |
| word_count_subset |   perfect    |   1   |  1.51448626703  |  0.049861952294 |
| word_count_subset |     love     |   1   |  1.36543549368  | 0.0303546295109 |
|    (intercept)    |     None     |   1   |   1.2995449552  | 0.0120888541331 |
| word_count_subset |     easy     |   1   |  1.19366189833  |  0.029288869202 |
| word_count_subset |    great     |   1   |  0.94469126948  | 0.0209509926591 |
| word_count_subset |    little    |   1   |  0.520628636025 | 0.0214691475665 |
| word_count_subset |     well     |   1   |  0.504256746398 |  0.021381300631 |
| word_count_subset |     able     |   1   |  0.191438302295 | 0.0337581955697 |
| word_count_subset |     ol

**Quiz Question**: Consider the coefficients of **simple_model**. There should be 21 of them, an intercept term + one for each word in **significant_words**. How many of the 20 coefficients (corresponding to the 20 **significant_words** and *excluding the intercept term*) are positive for the `simple_model`?

In [66]:
positives = 0
positive_significant_words = list()
for i in range(21):
    if simple_model.coefficients[i]["value"] >0:
        positives += 1
        positive_significant_words.append(simple_model.coefficients[i]["index"])

print(positives-1)

10


**Quiz Question**: Are the positive words in the **simple_model** (let us call them `positive_significant_words`) also positive words in the **sentiment_model**?

In [ ]:
print(positive_significant_words)

sentiment = sentiment_model.coefficients.select_columns(["index" , "value"])
sent_pos = sentiment[sentiment["value"]>0]

sent_pos_index = list()

a=0

print(sent_pos[0]["index"])

for i in range(len(sent_pos)):
    sent_pos_index.append(sent_pos[i]["index"])
    if(sent_pos[i]["index"]) in positive_significant_words:
        print(sent_pos[i]["index"])
print(sent_pos_index)

     
        


In [100]:

sentiment = sentiment_model.coefficients.select_columns(["index" , "value"])
sent_pos = sentiment[sentiment["value"]>0]


In [103]:
for i in range(len(sent_pos)):
    sent_pos_index.append(sent_pos[i]["index"])
    if(sent_pos[i]["index"]) in positive_significant_words:
        print(sent_pos[i]["index"])

None
love
well
loves
little
easy
great
able
perfect
old
car


KeyboardInterrupt: 

# Comparing models

We will now compare the accuracy of the **sentiment_model** and the **simple_model** using the `get_classification_accuracy` method you implemented above.

First, compute the classification accuracy of the **sentiment_model** on the **train_data**:

In [104]:
get_classification_accuracy(sentiment_model, train_data, train_data['sentiment'])

0.979440247046831

Now, compute the classification accuracy of the **simple_model** on the **train_data**:

In [105]:
get_classification_accuracy(simple_model, train_data, train_data['sentiment'])

0.8668150746537147

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TRAINING set?

Now, we will repeat this exercise on the **test_data**. Start by computing the classification accuracy of the **sentiment_model** on the **test_data**:

In [106]:
get_classification_accuracy(sentiment_model, test_data, test_data['sentiment'])

0.9145368370530358

Next, we will compute the classification accuracy of the **simple_model** on the **test_data**:

In [107]:
get_classification_accuracy(simple_model, test_data, test_data['sentiment'])

0.8693004559635229

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TEST set?

## Baseline: Majority class prediction

It is quite common to use the **majority class classifier** as the a baseline (or reference) model for comparison with your classifier model. The majority classifier model predicts the majority class for all data points. At the very least, you should healthily beat the majority class classifier, otherwise, the model is (usually) pointless.

What is the majority class in the **train_data**?

In [108]:
num_positive  = (train_data['sentiment'] == +1).sum()
num_negative = (train_data['sentiment'] == -1).sum()
print num_positive
print num_negative

112164
21252


Now compute the accuracy of the majority class classifier on **test_data**.

**Quiz Question**: Enter the accuracy of the majority class classifier model on the **test_data**. Round your answer to two decimal places (e.g. 0.76).

In [109]:
num_positive_test  = (test_data['sentiment'] == +1).sum()
num_negative_test = (test_data['sentiment'] == -1).sum()
print num_positive_test
print num_negative_test
print(num_positive_test/(num_positive_test+num_negative_test))

28095
5241
0.842782577394


**Quiz Question**: Is the **sentiment_model** definitely better than the majority class classifier (the baseline)?